In [ ]:
import os
import traceback
from glob import glob
import toml

from tqdm.auto import tqdm
import cv2

Load `make_cam_gallery.toml`

In [ ]:
with open("make_cam_gallery.toml", mode="r") as f_reader:
    config = toml.load(f_reader)

column = config["layout"]["column"]

line_color = config["draw"]["drop_image"]["line"]["color"]
line_width = config["draw"]["drop_image"]["line"]["width"]

cam_weight = config["draw"]["cam_image"]["weight"]
replace_cam_color = config["draw"]["cam_image"]["replace_color"]["enable"]
replaced_colormap  = getattr(cv2, config["draw"]["cam_image"]["replace_color"]["colormap"])
text_correct_color   = config["draw"]["cam_image"]["text"]["color"]["correct"]
text_incorrect_color = config["draw"]["cam_image"]["text"]["color"]["incorrect"]
text_shadow_color    = config["draw"]["cam_image"]["text"]["color"]["shadow"]
text_font_style      = config["draw"]["cam_image"]["text"]["font_style"]
text_font_size       = config["draw"]["cam_image"]["text"]["font_size"] # if None, do auto-detection

load_dir_root = config["model"]["history_root"]
model_name    = config["model"]["model_name"]
model_history = config["model"]["history"]

Load `train_config.toml`

In [ ]:
load_dir = os.path.join(load_dir_root, model_name, model_history)
train_config_path = os.path.join(load_dir, r"train_config.toml")

with open(train_config_path, mode="r") as f_reader:
    train_config = toml.load(f_reader)

dataset_root       = os.path.normpath(train_config["dataset"]["root"])
dataset_name       = train_config["dataset"]["name"]
dataset_gen_method = train_config["dataset"]["gen_method"]
dataset_stdev      = train_config["dataset"]["stdev"]
dataset_param_name = train_config["dataset"]["param_name"]

Generate `path_vars`

In [ ]:
dataset_dir = os.path.join(dataset_root, dataset_name, dataset_gen_method, dataset_stdev, dataset_param_name)
cam_result_root = os.path.join(load_dir, "cam_result")

Run

In [ ]:
train_dir_img_paths = glob(os.path.normpath(f"{dataset_dir}/train/*/*/*.tiff"))

test_dir_img_paths = glob(os.path.normpath(f"{dataset_dir}/test/*/*/*.tiff"))

cam_result_img_paths = glob(os.path.normpath(f"{cam_result_root}/*/*/*.tiff"))

print(f'train_dir_img_paths {type(train_dir_img_paths)}: {len(train_dir_img_paths)}')
print(f'test_dir_img_paths {type(test_dir_img_paths)}: {len(test_dir_img_paths)}')
print(f'cam_result_img_paths {type(cam_result_img_paths)}: {len(cam_result_img_paths)}')

In [ ]:
def image_read_test(img_path_list, pbar_desc):
    
    for path in tqdm(img_path_list, desc=f"Image Read Test ({pbar_desc}) "):
        
        try:
            cv2.imread(path)
        except Exception as e:
            print(traceback.format_exc())
            print(path)

In [ ]:
image_read_test(train_dir_img_paths, "train")
image_read_test(test_dir_img_paths, "test")
image_read_test(cam_result_img_paths, "cam_result")